In [26]:
import keras
import numpy as np
from keras import datasets
from numpy.linalg import eig 
from numpy.linalg import pinv
import matplotlib.pyplot as plt
from numpy import cov
%matplotlib inline

In [2]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 28*28)


In [3]:
x_0 = x_train[y_train == 0]
x_1 = x_train[y_train == 1]
mean_0 = np.mean(x_0, axis=0)
mean_1 = np.mean(x_1, axis =0)


# splitting data 

In [4]:
scatter_mat = np.zeros((784, 784))

In [30]:
# class 0 
mean_0_col = mean_0.reshape((mean_0.shape[0], 1))
scatter_mat = np.zeros((784, 784))

class_sc = np.zeros((784,784))
for row in x_0:
    row_col = row.reshape((row.shape[0], 1))
    scatter_mat += (row_col-mean_0_col).dot((row_col-mean_0_col).T)
class_sc += scatter_mat

scatter_mat = np.zeros((784, 784))
mean_1_col = mean_1.reshape((mean_1.shape[0], 1))
for row_2 in x_1:
    row_col = row.reshape((row_2.shape[0], 1))
    scatter_mat += (row_col-mean_1_col).dot((row_col-mean_1_col).T)
class_sc += scatter_mat

In [31]:
overall_mean = np.mean(x_train, axis = 0)
scatter_btw = np.zeros((784, 784))

In [32]:
overall_mean_vec = overall_mean.reshape(784, 1) # make column vector
mean_0_col_vec = mean_0.reshape((784,1))
scatter_btw = len(x_0) * (mean_0_col_vec - overall_mean_vec).dot((mean_0_col_vec - overall_mean_vec).T)

mean_1_col_vec = mean_1.reshape((784,1))
scatter_btw += len(x_1) * (mean_1_col_vec - overall_mean_vec).dot((mean_1_col_vec - overall_mean_vec).T)


In [33]:
eig_vals, eig_vecs = np.linalg.eig(pinv(class_sc).dot(scatter_btw))

